In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical

# Load the dataset
data = pd.read_csv('eurusd_final_dataset.csv')

# I am dropping because I don't want to use these features in the model currently
X = data.drop(['label', 'Date_Time', 'hour', 'dayofweek', 'mins_into_m15', 'frac_into_m15'], axis=1)
y = data['label']

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(64, 64), activation='relu', alpha=0.0001, random_state=42, max_iter=500)
mlp.fit(X_train, y_train)

y_pred_baseline = mlp.predict(X_test)
print("MLPClassifier Report:")
print(classification_report(y_test, y_pred_baseline))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_baseline))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Convert labels to categorical
y_train_cat = to_categorical(y_train, num_classes=5)
y_test_cat = to_categorical(y_test, num_classes=5)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train_cat, epochs=20, batch_size=32, validation_split=0.1, verbose=1)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Predict
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Metrics
print("Keras Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Per-class Precision/Recall/F1
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
for i in range(5):
    print(f"Class {i} — Precision: {prec[i]:.2f}, Recall: {rec[i]:.2f}, F1: {f1[i]:.2f}")


In [ ]:
# Simulated trade logic example: TP = +2, SL = -1 per class prediction
profit_rules = {0: 2, 1: -1, 2: 2, 3: -1, 4: 0}

# Assume true class is the correct "direction" — reward if prediction matches
profits = [profit_rules[p] if p == t else -1 for p, t in zip(y_pred, y_test)]
total_profit = sum(profits)

print(f"Simulated Profit: {total_profit}")
